In [2]:
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

DATA_PATH = "../../Datasets/MAMS-ATSA/Downsampled/test/test_single_row.csv"
TRAIN_PATH = "../../Datasets/MAMS-ATSA/Downsampled/train/train_single_row.csv"
OUTPUT_PATH = "../../Datasets/MAMS-ATSA/Downsampled/test/test_single_row_few_shot.csv"




In [3]:
def find_nearest_neighbors(df, target_sentence, K=3):
    df = df.drop_duplicates(subset='text_id')
    # Step 2: Vectorize the sentences in the DataFrame
    vect = TfidfVectorizer()
    transformed = vect.fit_transform(df['text'])

    # Vectorize the new sentence using the existing vectorizer
    new_sentence_vec = vect.transform([target_sentence])

    # Step 3: Compute cosine similarity between the new sentence and all DataFrame sentences
    cosine_similarities = cosine_similarity(new_sentence_vec, transformed).flatten()

    nearest_neighbor_ind = np.argsort(cosine_similarities)[-K:]
    nearest_neighbors = df.iloc[nearest_neighbor_ind]
    similarity_scores = cosine_similarities[nearest_neighbor_ind]


    return_df = nearest_neighbors.copy()
    return_df['og_ind'] = nearest_neighbor_ind
    return_df['Similarity_Score'] = similarity_scores
    return_df.reset_index(drop=True, inplace=True)


    examples_sentences = [] 
    for i in range(K):
        example_sentence = "Input: \"" + return_df['text'][i] + "\"\nTerm: \""+ return_df['term'][i] + "\"\n"+ "Output: \"" + return_df['polarity'][i] + "\"\n"
        examples_sentences.append(example_sentence)

    return return_df, examples_sentences

In [4]:
df = pd.read_csv(DATA_PATH)
df_train = pd.read_csv(TRAIN_PATH)

df['example1_term_id'] = None
df['example1_prompt'] = None
df['example1_sim_score'] = None
df['example2_term_id'] = None
df['example2_prompt'] = None
df['example2_sim_score'] = None
df['example3_term_id'] = None
df['example3_prompt'] = None
df['example3_sim_score'] = None

print(df_train.head())
print(df.head())


   text_id  term_id                                               text  \
0        0        0  The decor is not special at all but their food...   
1        0        1  The decor is not special at all but their food...   
2        0        2  The decor is not special at all but their food...   
3        1        3  when tables opened up, the manager sat another...   
4        1        4  when tables opened up, the manager sat another...   

      term  polarity  
0    decor  negative  
1     food  positive  
2   prices  positive  
3   tables   neutral  
4  manager  negative  
   text_id  term_id                                               text  \
0        0        0  Amusing details distinguish desserts, from dul...   
1        0        1  Amusing details distinguish desserts, from dul...   
2        0        2  Amusing details distinguish desserts, from dul...   
3        0        3  Amusing details distinguish desserts, from dul...   
4        1        4  The server was so busy the

In [5]:
def fill_examples(df, df_train):
    for i in range(len(df)):
        target = df['text'][i]
        example_df, example_sentences = find_nearest_neighbors(df_train, target, K = 3)
        for j in range(len(example_df)):
            df.loc[i, 'example{}_term_id'.format(j+1)] = example_df['term_id'][j]
            df.loc[i, 'example{}_prompt'.format(j+1)] = example_sentences[j]
            df.loc[i, 'example{}_sim_score'.format(j+1)] = example_df['Similarity_Score'][j]
        print(i)
    return df

df = fill_examples(df, df_train)

print(df.head())


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
df

,text_id,term_id,text,term,polarity,example1_term_id,example1_prompt,example1_sim_score,example2_term_id,example2_prompt,example2_sim_score,example3_term_id,example3_prompt,example3_sim_score
0,0,0,"Amusing details distinguish desserts, from dul...",desserts,positive,9500,"Input: ""Food is mediocre, except for Mushroom ...",0.152772,5385,"Input: ""It was the WORST Sangria ever, and the...",0.162247,1320,"Input: ""The overpriced ice cream is delicious ...",0.166906
1,0,1,"Amusing details distinguish desserts, from dul...",dulce de leche ice-cream,neutral,9500,"Input: ""Food is mediocre, except for Mushroom ...",0.152772,5385,"Input: ""It was the WORST Sangria ever, and the...",0.162247,1320,"Input: ""The overpriced ice cream is delicious ...",0.166906
2,0,2,"Amusing details distinguish desserts, from dul...",chocolate sauce tic-tac-toe,neutral,9500,"Input: ""Food is mediocre, except for Mushroom ...",0.152772,5385,"Input: ""It was the WORST Sangria ever, and the...",0.162247,1320,"Input: ""The overpriced ice cream is delicious ...",0.166906
3,0,3,"Amusing details distinguish desserts, from dul...",poached pineapple,neutral,9500,"Input: ""Food is mediocre, except for Mushroom ...",0.152772,5385,"Input: ""It was the WORST Sangria ever, and the...",0.162247,1320,"Input: ""The overpriced ice cream is delicious ...",0.166906
4,1,4,The server was so busy the night we visited th...,server,negative,1211,"Input: ""The server also forgot about our desse...",0.354028,8639,"Input: ""the 3rd however, the waiter forgot to ...",0.41081,10792,"Input: ""We waited an hour for our appetizers, ...",0.416744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,169,446,The server came by only once to pour additiona...,server,negative,10075,"Input: ""The waitress came to our table once fo...",0.252963,4290,"Input: ""The Server took the order and disappea...",0.264256,8702,"Input: ""Service was atrocious: appetizers came...",0.268229
447,169,447,The server came by only once to pour additiona...,wine,neutral,10075,"Input: ""The waitress came to our table once fo...",0.252963,4290,"Input: ""The Server took the order and disappea...",0.264256,8702,"Input: ""Service was atrocious: appetizers came...",0.268229
448,169,448,The server came by only once to pour additiona...,the table,neutral,10075,"Input: ""The waitress came to our table once fo...",0.252963,4290,"Input: ""The Server took the order and disappea...",0.264256,8702,"Input: ""Service was atrocious: appetizers came...",0.268229
449,169,449,The server came by only once to pour additiona...,fish,neutral,10075,"Input: ""The waitress came to our table once fo...",0.252963,4290,"Input: ""The Server took the order and disappea...",0.264256,8702,"Input: ""Service was atrocious: appetizers came...",0.268229


In [32]:
df.to_csv(OUTPUT_PATH, index = True)